<a href="https://colab.research.google.com/github/s-caro/psychic-broccoli/blob/main/Spring_and_flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install dwave-system

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
import math 
import dimod
from dimod import ConstrainedQuadraticModel, Binary, quicksum, Real, SampleSet, Integer, ExactCQMSolver
from dwave.system import LeapHybridCQMSampler

In [ ]:
dimod.REAL_INTERACTIONS =True

In [ ]:
class Variables:
    """Class that collects all CQM model variables for the Tutte's barycenter method
    Args:
        nodes: the nodes of the graph considered
        lower_bound: the minumum value for the coordinate of a node
        upper_bound: the maximum value for the coordinate of a node
    """

    def __init__(self, nodes: nx.Graph.nodes, lowerBound: float, upperBound: float):
        self.x = {i: Integer(f'x_{i}',lower_bound = lowerBound, upper_bound = upperBound) for i in nodes}
        self.y = {i: Integer(f'y_{i}',lower_bound = lowerBound, upper_bound = upperBound) for i in nodes}

In [ ]:
def _add_variable(cqm: ConstrainedQuadraticModel, lowerBound: float, upperBound: float, g: nx.Graph()):
    for i in G.nodes():
        cqm.add_variable('INTEGER',f'x_{i}',lower_bound = lowerBound, upper_bound = upperBound)
        cqm.add_variable('INTEGER',f'y_{i}',lower_bound = lowerBound, upper_bound = upperBound)

In [ ]:
def _add_constraint(cqm: ConstrainedQuadraticModel, vars: Variables, g: nx.Graph(), l: float):
    """constraint on the x and y position of the nodes, the sum of all forces acting on a point must be zero
    Args:
        cqm: the model
        vars: the x and y coordinates of the points
        g: the graph that we want to draw
    """
    l_1 = l**(-1)
    for u in g.nodes:
        nodes = list(set(g.nodes)-set([u]))
        F_rep_x = quicksum((-(l**2))*(vars.x[v]-vars.x[u])*((vars.x[v]-vars.x[u])**2+(vars.y[v]-vars.y[u])**(2)) for v in nodes)
        F_att_x = quicksum((math.sqrt((vars.x[u]-vars.x[v])**2+(vars.y[u]-vars.y[v])**2))*(vars.x[v]-vars.x[u])*l_1 for v in g.neighbors(u))
        cqm.add_constraint((F_rep_x + F_att_x)==0, label=f'x_constraint_node_{u}')
        #cqm.add_constraint((quicksum((-l**2)*(vars.y[v]-vars.y[u])*((vars.x[v]-vars.x[u])**2+(vars.y[v]-vars.y[u])**(2)) for v in nodes) + quicksum((math.sqrt((vars.x[u]-vars.x[v])**2+(vars.y[u]-vars.y[v])**2))*(vars.y[v]-vars.y[u])*(l**(-1)) for v in g.neighbors(u)))==0, label=f'y_constraint_node_{u}')

        #cqm.add_constraint((g.degree(v)*vars.x[v]-quicksum(vars.x[u] for u in g.neighbors(v)))==0,label=f'x_constraint_node_{u}')
        #cqm.add_constraint((g.degree(v)*vars.y[v]-quicksum(vars.y[u] for u in g.neighbors(v)))==0,label=f'y_constraint_node_{u}')

In [ ]:
def build_cqm(vars: Variables, g: nx.Graph(), upperBound: int, C: int) -> ConstrainedQuadraticModel:
    """objective function of the problem, minimize the distance of each point from the barycenter position
    Args:
        vars: the x and y coordinates of the points
        g: the graph that we want to draw
        fixed_points: a list of tuple, each contains the node, the x-coordinate, the y-coordinate where the node needs to be fixed
    
    Returns:
        A ''dimod.CQM'' object that defines the Titto's barycenter method
    """
    cqm = ConstrainedQuadraticModel()
    _add_variable(cqm, 0, upperBound, g)
    l = C*math.sqrt(upperBound**2/len(list(g.nodes)))
    _add_constraint(cqm, vars, g, l)
    
    
    #print(cqm.objective)
    print(cqm.constraints)
    
    return cqm

In [ ]:
num_nodes = 4
G = nx.from_edgelist([(0,1),(1,2),(2,0),(0,3),(1,3),(2,3)])
#G = nx.from_edgelist([(0,1),(0,4),(0,3),(1,5),(1,2),(2,3),(2,6),(3,7),(4,5),(4,7),(5,6),(6,7)])
#G = nx.from_edgelist([(0,1),(0,2),(0,3),(0,4),(0,6),(1,3),(1,4),(1,2),(1,5),(2,3),(2,5),(2,6),(3,4),(3,5),(3,6)])
#G =nx.from_edgelist([(0,1),(0,7),(0,4),(1,9),(1,2),(2,3),(2,19),(3,4),(3,17),(4,5),(5,6),(5,16),(6,7),(6,13),(7,8),(8,9),(8,12),(9,10),(10,11),(10,19),(11,12),(11,15),(12,13),(13,14),(14,15),(14,16),(15,18),(16,17),(17,18),(18,19)])
#G = nx.from_edgelist([(0,1),(0,2),(0,3),(0,4),(0,5),(0,6),(1,2),(1,3),(2,3),(2,4),(2,5),(2,6),(3,4),(4,5),(5,6)])
#G=nx.from_edgelist([(0,1),(0,10),(0,9),(1,2),(1,11),(2,3),(2,11),(3,4),(3,12),(4,5),(4,12),(5,6),(5,13),(6,7),(6,13),(7,8),(7,14),(8,9),(8,14),(9,10),(10,15),(11,17),(12,18),(13,19),(14,16),(15,16),(15,17),(16,19),(17,18),(18,19)])
upperBound = 5
lowerBound = 0
C = 1
vars = Variables(G.nodes(), lowerBound, upperBound)
cqm = build_cqm(vars, G, upperBound, C)